In [1]:
!pip install --upgrade pandas

     |████████████████████████████████| 10.0MB 3.5MB/s eta 0:00:01    |█████████████▎                  | 4.2MB 3.5MB/s eta 0:00:02
  Found existing installation: pandas 0.19.2
    Uninstalling pandas-0.19.2:
      Successfully uninstalled pandas-0.19.2
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pyarrow

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
from numpy import nan as NA
import os
import gc
from pandas.tseries.offsets import MonthEnd

from azure.storage.blob import BlockBlobService
import pyarrow.parquet as pq
from io import StringIO
from io import BytesIO

In [4]:
YearMonth = '2018-01-01'

In [5]:
#Azure account credentials
azure_storage_account_name = "agedatasciencecocstorage"
azure_storage_account_key = "V/FXG+lIWtYc854wbRA8BxGCLGJijS0NDwLvNHfmAxqRopAL3WHWGIQoHxpArG9lZoX+iY+dvtPYoB379w4MIw=="

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("You must provide a name and key for an Azure Storage account")

In [6]:
# First, we need to connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)
# There are also methods to list containers and blobs
#containers = blob_service.list_containers()
blobs = blob_service.list_blobs('banking-dm')

In [7]:
# List the blobs in the container.
print("\nList blobs in the container")
#generator = block_blob_service.list_blobs(container_name)
for blob in blobs:
    print("\t Blob name: " + blob.name)


List blobs in the container
	 Blob name: ailiev/Balances_Master_2018-01-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-02-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-03-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-04-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-05-01.parquet.gzip
	 Blob name: ailiev/Balances_Master_2018-06-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-01-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-02-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-03-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-04-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-05-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-06-01.parquet.gzip
	 Blob name: ailiev/sample.txt
	 Blob name: data/Customer_economic_sector_lookup.csv
	 Blob name: data/Customer_information_2018-01-01.parquet.gzip
	 Blob name: data/Customer_information_2018-02-01.parquet.gzip
	 Blob name: dat

### Read Data

In [8]:
container_name = "banking-dm"
blob_name_parquet = 'data/Customer_information_' + YearMonth + '.parquet.gzip'
blob_name = 'data/Customer_economic_sector_lookup.csv'

In [9]:
byte_stream = BytesIO()
try:
   blob_service.get_blob_to_stream(container_name=container_name, blob_name=blob_name_parquet, stream=byte_stream)
   cust_info = pq.read_table(source=byte_stream).to_pandas()
   # Do work on df ...
finally:
   # Add finally block to ensure closure of the stream
   byte_stream.close()

In [10]:
cust_info.head()

,Customer_ID,YearMonth,Birth_date,Customer_type,Customer_category,Gender,Marital_Status,Company_age,Customer_economic_sector,Customer_economic_branch
0,1,2018-01-01,1971-09-10,Individual,None,F,Divorced,None,ECO03,BR160
1,2,2018-01-01,1974-12-01,Professional,None,M,Married,None,ECO06,BR09
2,3,2018-01-01,1992-05-21,Individual,None,M,Married,None,ECO13,BR145
3,4,2018-01-01,None,Small Business,Retail,None,None,1977-11-08,ECO08,BR14
4,5,2018-01-01,1978-06-05,Individual,None,M,Single,None,ECO02,BR171


In [11]:
cust_info.shape

(100000, 10)

In [12]:
pd.value_counts(cust_info['Gender'], dropna=False)

M      44022
F      40602
NaN    15376
Name: Gender, dtype: int64

In [ ]:
#Data Audit
#Count + Unique Count + Missing values by column; Statistic summary for numeric variables; Frequencies for non-numeric variables
def data_audit(df, excel_name = 'Data_Audit.xlsx'):
    #open excel writer
    writer = pd.ExcelWriter(str(output_path + excel_name))
    #sum up the missing values by columns
    missing = pd.Series(df.isnull().sum(), name='missing')
    #use describe function on all columns, transpose it and join the missings summary from above
    describe = df.describe(include='all').T.join(missing)[['count', 'unique', 'missing']]
    #write to excel
    describe.to_excel(writer, sheet_name="Summary", na_rep='NA')
    #find categorical/numerical columns
    categorical_columns = [col for col in df.columns if df[col].dtype == 'O']
    numerical_columns = [col for col in df.columns if df[col].dtype != 'O']
    #export describe statistics only for numeric variables
    missing_num = pd.Series(df[numerical_columns].isnull().sum(), name='missing')
    df[numerical_columns].describe(include='all').T.join(missing_num).to_excel(writer, sheet_name="Numeric", na_rep='NA')
    #loop all categorical variables, value counts, append Total and export to excel sheet
    for value in categorical_columns:
        dictionary = {}
        dictionary = {'Total': df[value].value_counts(dropna=False).sum()}
        df[value].value_counts(dropna=False).append(pd.Series(dictionary)).to_excel(writer, sheet_name=value, na_rep='NA')
    #close excel writer
    writer.save()
    del missing, describe, missing_num
    gc.collect()

In [ ]:
#data_audit(cust_info, 'Cust_Info_Data_Audit.xlsx')

In [13]:
#read csv file
blobstring = blob_service.get_blob_to_text(container_name, blob_name).content
eco_sector = pd.read_csv(StringIO(blobstring), header=0, sep=';')

In [14]:
eco_sector

,Sector,Customer_economic_sector_code
0,Architecture and Construction,ECO01
1,Agriculture and Food,ECO02
2,Arts and Communications,ECO03
3,Business Administration,ECO04
4,Education and Training,ECO05
5,Finance,ECO06
6,Government and Public,ECO07
7,Health and Science,ECO08
8,Hospitality and Tourism,ECO09
9,Human Services,ECO10


In [15]:
merged = cust_info.merge(eco_sector, how='left', left_on='Customer_economic_sector', right_on='Customer_economic_sector_code')

In [16]:
merged.head()

,Customer_ID,YearMonth,Birth_date,Customer_type,Customer_category,Gender,Marital_Status,Company_age,Customer_economic_sector,Customer_economic_branch,Sector,Customer_economic_sector_code
0,1,2018-01-01,1971-09-10,Individual,None,F,Divorced,None,ECO03,BR160,Arts and Communications,ECO03
1,2,2018-01-01,1974-12-01,Professional,None,M,Married,None,ECO06,BR09,Finance,ECO06
2,3,2018-01-01,1992-05-21,Individual,None,M,Married,None,ECO13,BR145,Manufacturing,ECO13
3,4,2018-01-01,None,Small Business,Retail,None,None,1977-11-08,ECO08,BR14,Health and Science,ECO08
4,5,2018-01-01,1978-06-05,Individual,None,M,Single,None,ECO02,BR171,Agriculture and Food,ECO02


In [17]:
del merged['Customer_economic_sector']
del merged['Customer_economic_sector_code']

In [18]:
merged.head()

,Customer_ID,YearMonth,Birth_date,Customer_type,Customer_category,Gender,Marital_Status,Company_age,Customer_economic_branch,Sector
0,1,2018-01-01,1971-09-10,Individual,None,F,Divorced,None,BR160,Arts and Communications
1,2,2018-01-01,1974-12-01,Professional,None,M,Married,None,BR09,Finance
2,3,2018-01-01,1992-05-21,Individual,None,M,Married,None,BR145,Manufacturing
3,4,2018-01-01,None,Small Business,Retail,None,None,1977-11-08,BR14,Health and Science
4,5,2018-01-01,1978-06-05,Individual,None,M,Single,None,BR171,Agriculture and Food


In [ ]:
pd.value_counts(merged['YearMonth'], dropna=False)

In [ ]:
merged.dtypes

In [19]:
#merged['YearMonth'] = pd.to_datetime(merged['YearMonth'], format="%Y-%m-%d") + MonthEnd(1)
merged['YearMonth'] = pd.to_datetime(merged['YearMonth'], format="%Y-%m-%d")  + MonthEnd(0)
merged['Birth_date'] = pd.to_datetime(merged['Birth_date'])
merged['Company_age'] = pd.to_datetime(merged['Company_age'])

In [21]:
merged['Age_Months'] = np.where(pd.notnull(merged['Birth_date']), 
                         (merged['YearMonth'] - merged['Birth_date'])/np.timedelta64(1,'M'),
                        (merged['YearMonth'] - merged['Company_age'])/np.timedelta64(1,'M'))

In [22]:
merged['Age_Months'] = merged['Age_Months'].astype('int')

In [23]:
del merged['Birth_date']
del merged['Company_age']

In [24]:
merged.head()

,Customer_ID,YearMonth,Customer_type,Customer_category,Gender,Marital_Status,Customer_economic_branch,Sector,Age_Months
0,1,2018-01-31,Individual,None,F,Divorced,BR160,Arts and Communications,556
1,2,2018-01-31,Professional,None,M,Married,BR09,Finance,518
2,3,2018-01-31,Individual,None,M,Married,BR145,Manufacturing,308
3,4,2018-01-31,Small Business,Retail,None,None,BR14,Health and Science,482
4,5,2018-01-31,Individual,None,M,Single,BR171,Agriculture and Food,475


In [26]:
pd.value_counts(merged['Gender'], dropna=False)

M      44022
F      40602
NaN    15376
Name: Gender, dtype: int64

In [27]:
pd.value_counts(merged[merged['Customer_type']=='Individual']['Customer_type'], dropna=False)

Individual    84121
Name: Customer_type, dtype: int64

In [28]:
merged = merged[merged['Customer_type']=='Individual']

In [29]:
merged.head()

,Customer_ID,YearMonth,Customer_type,Customer_category,Gender,Marital_Status,Customer_economic_branch,Sector,Age_Months
0,1,2018-01-31,Individual,None,F,Divorced,BR160,Arts and Communications,556
2,3,2018-01-31,Individual,None,M,Married,BR145,Manufacturing,308
4,5,2018-01-31,Individual,None,M,Single,BR171,Agriculture and Food,475
5,6,2018-01-31,Individual,None,M,Married,BR67,Hospitality and Tourism,290
6,7,2018-01-31,Individual,None,M,Other,BR152,Health and Science,437


In [30]:
pd.value_counts(merged['Customer_category'], dropna=False)

NaN    84121
Name: Customer_category, dtype: int64